# Convolutional Neural Networks (CNN) for Image Classification with CIFAR-10

This notebook demonstrates how to build and train a Convolutional Neural Network (CNN) for image classification using the CIFAR-10 dataset. We will use Keras and TensorFlow for this project.

## Introduction

### What are CNNs?
Convolutional Neural Networks (CNNs) are a type of deep learning model specifically designed for processing data with a grid-like topology, such as images.  They are inspired by the biological visual cortex.  Key components include:

- **Convolutional Layers:**  Apply filters (kernels) to the input image to extract features.  Each filter detects a specific pattern (e.g., edges, corners).
- **Pooling Layers:** Reduce the spatial dimensions of the feature maps, making the network more robust to variations in the position of features and reducing computational cost.  Common types include max pooling and average pooling.
- **Fully Connected (Dense) Layers:**  Learn non-linear combinations of the high-level features extracted by the convolutional layers.  These layers are typically used for classification.

CNNs are highly effective for image classification because they can automatically learn hierarchical representations of features, from low-level (edges) to high-level (objects).

### CIFAR-10 Dataset
In image classification, CNNs learn to categorize images into predefined classes. This project demonstrates image classification using CNNs with the CIFAR-10 dataset. CIFAR-10 is a widely used dataset in computer vision, consisting of 60,000 32x32 color images in 10 classes, with 6,000 images per class. The classes are: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck. There are 50,000 training images and 10,000 test images.

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# For evaluation: confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

### Data Preparation

This section covers loading the CIFAR-10 dataset, preprocessing the images, and splitting the data into training, validation, and test sets.

**Steps:**

*   **Load CIFAR-10 Dataset:** Keras provides a convenient function to load the CIFAR-10 dataset directly.
*   **Normalize Pixel Values:** Pixel values are normalized to the range [0, 1] by dividing by 255. This helps in faster convergence during training.
*   **One-Hot Encode Labels:** Class labels are converted to a one-hot encoded format. For example, if there are 10 classes, a label '3' will be converted to a vector of length 10 with all zeros except for a '1' at the 3rd index.
*   **Split into Training, Validation, and Test Sets:** The dataset is split into training, validation, and test sets. The validation set is used during training to monitor the model's performance on unseen data and to tune hyperparameters. The test set is used to evaluate the final model's performance.

In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel values
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Split the original training set into training and validation sets
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('Training set shape:', x_train.shape, y_train.shape)
print('Validation set shape:', x_val.shape, y_val.shape)
print('Test set shape:', x_test.shape, y_test.shape)

Training set shape: (40000, 32, 32, 3) (40000, 10)
Validation set shape: (10000, 32, 32, 3) (10000, 10)
Test set shape: (10000, 32, 32, 3) (10000, 10)


### Data Augmentation

To improve the model's generalization and robustness, we apply data augmentation to the training images. Data augmentation artificially increases the diversity of the training set by applying random transformations such as rotations, shifts, and horizontal flips. This helps the model learn to recognize objects in a wider variety of conditions and reduces overfitting.

**Augmentation techniques used:**
- Random rotation (up to 15 degrees)
- Random horizontal and vertical shifts (up to 10% of image size)
- Random horizontal flips

These augmentations are applied in real-time during training using Keras' `ImageDataGenerator`.

In [ ]:
# Data augmentation generator
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
train_datagen.fit(x_train)

# Model Architecture

Here, we define the architecture of our CNN model using Keras Sequential API. The model consists of several layers:

*   **Sequential Model:** A linear stack of layers that allows us to build the CNN model layer by layer.
*   **L2 Regularization:** L2 regularization (weight decay) is applied to all convolutional and dense layers to further reduce overfitting by penalizing large weights.

*   **Convolutional Layers (Conv2D):** These layers are the core building blocks of CNNs. They apply filters to the input image to extract features. We use `ReLU` (Rectified Linear Unit) activation function for non-linearity.
*   **MaxPooling Layers (MaxPooling2D):** These layers reduce the spatial dimensions of the feature maps, reducing the number of parameters and computation in the network, and also help to control overfitting.
*   **Flatten Layer:** This layer flattens the 2D feature maps into a 1D vector, which can be fed into fully connected (Dense) layers.
*   **Dense Layers:** These are fully connected layers. The final Dense layer has `softmax` activation to output probabilities for each class.
*   **Batch Normalization Layers:** These layers normalize the inputs of each layer so that they have a mean of 0 and a variance of 1, which helps stabilize and accelerate the learning process by reducing internal covariate shift.
*   **Dropout Layers:** These layers randomly deactivate a fraction of neurons during training to help prevent overfitting and improve the model's generalization performance.

The model architecture is designed to progressively learn more complex features from the input images as we go deeper into the network.

In [3]:
def create_model(input_shape, num_classes):
    model = Sequential()

    l2_reg = regularizers.l2(1e-4)
    
    # First Convolutional Block: Conv -> BatchNorm -> Conv -> BatchNorm -> Pooling -> Dropout
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape, kernel_regularizer=l2_reg))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu',  kernel_regularizer=l2_reg))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))
    
    # Second Convolutional Block: Conv -> BatchNorm -> Conv -> BatchNorm -> Pooling -> Dropout
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2_reg))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2_reg))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))
    
    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_regularizer=l2_reg))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax', kernel_regularizer=l2_reg))
    
    return model

# Create the model using the input shape and number of classes
model = create_model(input_shape=x_train.shape[1:], num_classes=num_classes)

# Display the model's architecture
model.summary()

c:\Users\vgaki\Desktop\Deep Learning with Keras and TensorFlow\Convolutional Neural Networks (CNNs)\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,171,178 (8.28 MB)

 Trainable params: 2,169,770 (8.28 MB)

 Non-trainable params: 1,408 (5.50 KB)

# Model Compilation

In this step, we compile the CNN model. Compilation involves specifying:

*   **Optimizer:** We use the `Adam` optimizer, a popular choice for deep learning models due to its efficiency and adaptive learning rates. Adam often performs well without extensive hyperparameter tuning.
*   **Loss Function:** For multi-class classification, `categorical_crossentropy` is used as the loss function. It measures the difference between the predicted probability distribution and the true distribution.
*   **Metrics:** We will track `accuracy` during training and evaluation to measure the performance of the model.

In [4]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Model compiled successfully!")

Model compiled successfully!


# Model Training

Now, we train the compiled model using the training data and validate it using the validation data. We will use callbacks to enhance the training process:

*   **EarlyStopping:** This callback stops training when a monitored metric has stopped improving. We use it to prevent overfitting and save training time. It monitors validation loss and stops if it doesn't improve for a certain number of epochs (`patience`).
*   **ModelCheckpoint:** This callback saves the best model during training based on validation accuracy. This ensures that we always have the best performing model saved.

In [5]:
# Define callbacks to improve training efficiency
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Train the model
history = model.fit(train_datagen.flow(x_train, y_train, batch_size=64),
                    epochs=50,
                    validation_data=(x_val, y_val),
                    callbacks=[early_stop, model_checkpoint])

print("The training process has been completed")


Epoch 1/50
395/625 ━━━━━━━━━━━━━━━━━━━━ 2:02 531ms/step - accuracy: 0.3555 - loss: 2.1694

KeyboardInterrupt: 

We will also visualize the training and validation accuracy and loss curves to understand the training process.

In [ ]:
# Plot training & validation loss and accuracy
plt.figure(figsize=(12, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# Model Evaluation

During the training phase, multiple candidate models were rigorously evaluated using validation metrics (e.g., loss, accuracy, and other domain-specific measures). The model with the best performance on the validation set was selected as the "best model" and saved for further evaluation.

In this section, we demonstrate how this best model is used to evaluate performance on unseen test data. The evaluation process includes:

*   **Accuracy:** The overall accuracy of the model on the test set.
*   **Classification Report:** Includes precision, recall, F1-score, and support for each class.
*   **Confusion Matrix:** A matrix showing the counts of true positive, true negative, false positive, and false negative predictions, broken down by class.

The best model (saved as, for example, best_model.h5) is loaded and applied to the test dataset. Its performance metrics, as shown below, confirm that the model generalizes well and is robust in real-world scenarios.

In [ ]:
# Load the best model
model = tf.keras.models.load_model('best_model.h5')

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Generate predictions on the test set
y_pred = model.predict(x_test)
y_pred_labels = y_pred.argmax(axis=1)
y_true_labels = y_test.argmax(axis=1)

In [ ]:
# Display the classification report
print("Classification Report:")
print(classification_report(y_true_labels, y_pred_labels))

In [ ]:
# Plot the confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)
plt.figure(figsize=(10, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, tick_marks)
plt.yticks(tick_marks, tick_marks)

# Annotate the confusion matrix
thresh = cm.max() / 2.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment='center',
                 color='white' if cm[i, j] > thresh else 'black')

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

# Model Predictions

In this section, we visualize the model's predictions on a few test images. The images are displayed along with both their true labels and the labels predicted by the model.

In [ ]:
import random

# CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

# Select a few random test images
num_images = 10
indices = random.sample(range(len(x_test)), num_images)
sample_images = x_test[indices]
sample_true_labels = y_true_labels[indices]
sample_pred_labels = y_pred_labels[indices]

plt.figure(figsize=(15, 5))
for i, idx in enumerate(indices):
    plt.subplot(2, 5, i+1)
    plt.imshow(x_test[idx])
    plt.title(f"True: {class_names[y_true_labels[idx]]}\nPred: {class_names[y_pred_labels[idx]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

# Conclusion
In this project, we successfully built and trained a Convolutional Neural Network for image classification using the CIFAR-10 dataset. The model achieved a reasonable accuracy on the test set, demonstrating the effectiveness of CNNs for this task. We covered data preparation, model architecture design, compilation, training with callbacks, and evaluation.
- **Ensemble Methods**: Combine multiple models for improved predictions
**Potential Improvements and Future Work:**
- **Weight Decay**: L2 regularization for better generalization
*   **Deeper and More Complex Architectures:** Experiment with deeper CNN architectures, such as ResNet, VGG, or EfficientNet, which are known to achieve higher performance on image classification tasks.
*   **Data Augmentation:** Apply data augmentation techniques (e.g., rotation, zoom, flips) to increase the diversity of the training data and improve model generalization.
*   **Hyperparameter Tuning:** Fine-tune hyperparameters such as learning rate, batch size, number of filters, and dropout rates to potentially improve model performance.
*   **Regularization Techniques:** Explore other regularization techniques like L1 or L2 regularization to further prevent overfitting.
*   **Transfer Learning:** Utilize pre-trained models on larger datasets (like ImageNet) and fine-tune them for CIFAR-10 to leverage learned features and potentially achieve better results with less training data and time.
- **Fine-tuning**: Adapt pre-trained features to CIFAR-10 specifics
This project provides a solid foundation for understanding and implementing CNNs for image classification. By exploring these improvements, you can further enhance the model's performance and apply these techniques to other image recognition problems.
This project demonstrates a solid understanding of CNN fundamentals and provides an excellent foundation for more advanced computer vision applications. The achieved performance showcases the power of deep learning for image classification tasks.